In [ ]:
import pandas as pd
from workflow import Workflow
from IPython.display import display, Code, Markdown
import ipywidgets as widgets
import ipyvuetify as v
from ipyaggrid import Grid
import io
import contextlib

# Initialize the query processor
query_processor = Workflow()
query_processor.initialize()

# Cypher LLM Options
cypher_ll_options = [
    ("Groq", "llama3-70b-8192"),
    ("Groq", "mixtral-8x7b-32768"),
    ("Groq", "gemma2-9b-it"),
    ("Groq", "llama3-groq-70b-8192-tool-use-preview"),
    ("Ollama", "codegemma:instruct")
]

formatted_options = [f'{provider}: {model}' for provider, model in cypher_ll_options]

# Widgets for selecting models
cypher_llm_choice = v.Select(
    label='Select a Language Model',
    items=[{'text': opt, 'value': opt} for opt in formatted_options],
    v_model=formatted_options[0]
)

qa_llm_choice = v.Select(
    label="Choose the QA Model",
    items=[{'text': model, 'value': model} for model in ["llama3-70b-8192", "mixtral-8x7b-32768", "gemma2-9b-it", "llama3-groq-70b-8192-tool-use-preview"]],
    v_model="llama3-70b-8192"
)

# Text area for user query
user_query_input = v.Textarea(
    label="Enter your query in natural language:",
    outlined=True,
    rows=4,
    v_model=''
)

# Button to submit query
submit_button = v.Btn(children=["Submit"], color='primary', dark=True)

# Output area for displaying results
output_area = widgets.Output()

# Function to convert DataFrame to Vuetify DataTable items and headers
def dataframe_to_vuetify_table(df):
    headers = [{'text': col, 'value': col} for col in df.columns]
    items = df.to_dict(orient='records')
    return headers, items
    
# Function to process query
def on_submit_button_clicked(*args):
    with output_area:
        output_area.clear_output()  # Clear previous output
        user_query = user_query_input.v_model
        
        if user_query:
            provider, llm = cypher_llm_choice.v_model.split(": ")

            # Redirect stdout to avoid prints from external libraries showing up
            buffer = io.StringIO()
            with contextlib.redirect_stdout(buffer):
                result = query_processor.handle_query(user_query)

            # Display refined question
            if 'refined_question' in result:
                display(Markdown("### Automated Refined Question"))
                display(Markdown(result['refined_question']))
            
            # Display generated Cypher query
            if 'cypher_query' in result:
                display(Markdown("### Generated Cypher Query"))
                display(Code(result['cypher_query'], language='cypher'))
            
            # Display query result in a table using ipyaggrid
            if 'cypher_result' in result:
                display(Markdown("### Query Result"))
                df = pd.DataFrame(result['cypher_result'])
                headers, items = dataframe_to_vuetify_table(df)
                data_table = v.DataTable(headers=headers, items=items, style_='height: 400px; overflow-y: auto;')
                display(data_table)
            
            # Display final answer
            display(Markdown("### Final Answer"))
            display(Markdown(result['final_answer']))
        else:
            display(Markdown("**Error:** Please enter a query."))

# Link button click to function
submit_button.on_event('click', on_submit_button_clicked)

# Layout and display
container = v.Container(children=[
    v.Row(children=[v.Col(children=[v.Html(tag='h3', children=['Inspect your Java Project'])])]),
    v.Row(children=[
        v.Col(children=[cypher_llm_choice], cols=4),
        v.Col(children=[qa_llm_choice], cols=4),
    ]),
    v.Row(children=[user_query_input]),
    v.Row(children=[submit_button]),
    v.Row(children=[output_area])
])

# Display the app layout
display(container)
